# Functions for reading a graph from a file and writing to the file from the graph.

In [21]:
import numpy as np
import pandas as pd
import networkx as nx
import pickle

import matplotlib.pyplot as plt

In [22]:
def graph_from_input(filename):
    '''graph_from_input(str) --> nx.graph G, int source, np.ndarray[int] homes, dict locToIndex 
    Returns a graph created by reading the input file, with integer vertex labels
    Returns list of the home indices
    Returns a map from integer to the name associated with that node'''
    with open(filename, 'r') as f:
        G = nx.Graph()
        
        locToIndex = {} # maps location name to its index number
        homes = []
        lines = f.readlines()
        
        numLocations = int(lines[0])
        numTAs = int(lines[1])
        locations = lines[2].split()
    
        i = 0
        assert len(locations) == numLocations, "Number of locations must match specified value"
        for loc in locations:
            G.add_node(i)
            locToIndex[loc] = i
            i += 1
            
        TAhomes = lines[3].split()
        assert len(TAhomes) == numTAs, "Number of TA homes must match specified value"
        for home in TAhomes:
            homes.append(locToIndex[home])
        
        source = locToIndex[lines[4].strip()]
        
        row = 0
        for line in lines[5:]:
            line = line.split()
            for col in range(len(line)):
            
                if line[col] != 'x':  
                    G.add_edge(row, col)
                    weight = float(line[col])
                    G[row][col]['weight'] = weight
            row += 1
        
        indexToLoc = {v: k for k, v in locToIndex.items()}
        return G, source, homes, indexToLoc

In [23]:
def write_input_file(G, homes, filename):
    '''write_graph(nx.graph, np.ndarray, str) --> None
    Write a graph G to the file pointed to by filename.
    The file will be written in the input format as specified
    in the project spec.
    
    Note - This assumes homes[0] is the starting vertex OLD CONVENTION
    '''
    assert isinstance(G, nx.Graph) , "G must be a graph"
    assert isinstance(homes, np.ndarray) , "homes must be an np array of integers"
    assert isinstance(filename, str) , "filename must be a string"
    
    locations = G.nodes
    #print(locations)
    num_homes = len(homes) - 1
    with open(filename, 'w') as f:
        f.write(str(len(locations)) + "\n")
        f.write(str(num_homes) + "\n")
        
        for g in G.nodes:
            f.write("Building{} ".format(g))
        f.write('\n')
    
        for h in homes:
            f.write("Building{} ".format(h))
        f.write('\n')
        
        f.write("Building{} \n".format(homes[0])) # Starting location, usually Soda
        
        adjMat = nx.to_numpy_array(G)
        for row in adjMat:
            for elt in row:
                if elt == 0:
                    f.write('x ')
                else:    
                     f.write(str(round(elt, 5)) + ' ')
            f.write('\n')

In [24]:
def write_output_file(path, dropOffs, indexToName, filename):
    '''path is a list of integers that we follow in the car 
    dropOffs is a dictionary (int -> [home, home, ...] ) mapping nodes to the homes of the TAs that get off at that node 
    indexToName is a list of names corresponding to each index
    filename is a string filename that we write to  
    '''
    with open(filename, 'w') as f:
        for step in path:
            f.write(indexToName[step] + " ")
        
        f.write("\n")
        count = 0
        for key in dropOffs:
            if dropOffs[key] != []:
                count += 1
        f.write(str(count) + "\n")
        
        for key in dropOffs:
            if dropOffs[key] != []:
                f.write(indexToName[key] + " ")
                for elt in dropOffs[key]:
                    f.write(indexToName[elt] + " ");
                f.write("\n")

In [25]:

sizes = [50, 100, 200]
indexToName = {i : "Building{}".format(i) for i in range(200)}

for size in sizes:
    G = nx.readwrite.gpickle.read_gpickle('./inputGraphs/' + str(size) + '/tempG.pkl')
    with open('./inputgraphs/' + str(size) + '/homes.pkl', 'rb') as f: homes = pickle.load(f)

    write_input_file(G, homes, str(size) + ".in")

    with open('./inputgraphs/' + str(size) + '/where.pkl', 'rb') as f: dropOffs = pickle.load(f)
    with open('./inputgraphs/' + str(size) + '/final_path.pkl', 'rb') as f: path = pickle.load(f)
    
    write_output_file(path, dropOffs, indexToName, str(size) + ".out")


G, source, homes, locToIndex = graph_from_input("./inputs/1_50.in")
print(np.ndarray.tolist(nx.to_numpy_matrix(G)))
print(homes)
print(locToIndex)


[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1531.32, 0.0, 0.0, 0.0, 1565.13, 0.0, 0.0, 0.0, 0.0, 1331.54, 1191.07, 1291.06, 1460.14, 0.0, 0.0, 0.0, 1112.35, 0.0, 1122.74, 0.0, 1390.95, 921.103, 1425.14, 0.0, 858.663, 0.0, 1004.84, 0.0, 0.0, 0.0, 0.0, 1013.32, 0.0, 1524.7, 0.0, 0.0, 0.0, 1639.54, 0.0, 1511.15, 1280.08, 0.0, 0.0, 1362.36], [0.0, 0.0, 926.505, 686.964, 901.291, 937.588, 0.0, 1212.94, 0.0, 1218.92, 0.0, 0.0, 1069.33, 0.0, 0.0, 0.0, 557.692, 0.0, 0.0, 0.0, 1055.4, 889.488, 0.0, 0.0, 0.0, 812.672, 1138.55, 0.0, 0.0, 0.0, 1061.42, 683.918, 0.0, 887.522, 0.0, 0.0, 0.0, 1020.69, 0.0, 1059.56, 724.018, 871.447, 0.0, 826.058, 590.927, 0.0, 634.981, 0.0, 0.0, 1225.25], [0.0, 926.505, 0.0, 1058.08, 0.0, 0.0, 991.677, 0.0, 0.0, 958.245, 405.139, 0.0, 0.0, 0.0, 0.0, 0.0, 1050.19, 772.727, 0.0, 0.0, 846.962, 0.0, 0.0, 0.0, 0.0, 0.0, 526.623, 824.097, 0.0, 0.0, 0.0, 0.0, 889.361, 311.385, 0.0, 657.765, 0.0, 1114.19, 0.0, 720.521, 0.0, 658.965, 0.0, 0.0, 1311.01, 1083.62, 644.947, 0.0, 1042.03, 879